In [3]:
import os
import sys
import pandas as pd
import numpy as np
from ezchord import Chord
from mingus.core.notes import int_to_note

sys.path.append('..', '..')
from src.melody import Melody
from src.utils import get_chord_progressions, is_weakly_polyphonic, is_strongly_polyphonic, remove_strong_polyphony, remove_weak_polyphony

In [4]:
bpm = 4

cp = get_chord_progressions('..')['A Felicidade']

linear_chord_progression = []

for section in cp['sections']:
    linear_chord_progression += cp['progression'][section]

n_chord_prog_measures = int(len(linear_chord_progression) / bpm)

In [5]:
orig = pd.read_csv('../data/split_melody_data/v1.2/Real Book/A Felicidade -o-.csv', index_col=0)
orig['end_ticks'] = (orig['ticks'] + orig['duration'])
orig.sort_values('ticks', inplace=True)

In [6]:
impro = pd.read_csv('../data/split_melody_data/v1.2/JazzPage/A Felicidade -1-.csv', index_col=0)
impro['end_ticks'] = (impro['ticks'] + impro['duration'])
impro.sort_values('ticks', inplace=True)

In [7]:
def multiple_pitches_to_string(pitches):
    return "-".join(str(x) for x in pitches)

In [8]:
tpm = 48

def get_current_chord(row, linear_chord_progression):
    offset = row['measure'] + (row['offset'] / tpm)
    chord_idx = int(np.floor(offset * 4)) # TODO improve for 3/4

    return linear_chord_progression[chord_idx]

In [ ]:
duration_correction = 1

In [114]:
def extract_mono_time_step_encoding(impro, orig, linear_chord_progression):
    tpm = 48
    n_chord_prog_beats = len(linear_chord_progression)
    n_ticks = tpm * n_chord_prog_measures

    rows = []

    if is_strongly_polyphonic(impro):
        print('Impro SP')
        impro = remove_strong_polyphony(impro)
        
        if is_strongly_polyphonic(impro):
            raise Exception('Error!!! P')

    if is_weakly_polyphonic(impro):
        print('Impro WP')
        impro = remove_weak_polyphony(impro)
               

        if is_weakly_polyphonic(impro):
            raise Exception('Error!!! W')

    if is_strongly_polyphonic(orig):
        print('Original SP')
        
        orig = remove_strong_polyphony(orig)
        
        if is_strongly_polyphonic(impro):
            raise Exception('Error!!! P')
            
    if is_weakly_polyphonic(orig):
        print('Original WP')
        
        orig = remove_weak_polyphony(orig)

        if is_weakly_polyphonic(orig):
            raise Exception('Error!!! W')
        
    
    ticks = 0
    duration = 0
    for i, row in impro.iterrows():   
        diff = row['ticks'] - (duration + ticks)
        
        if diff > 1:                      
            rows.append({
                'diff': diff,
                'ticks': (ticks + duration),
                'offset': (ticks + duration) % tpm,
                
                'impro_pitch': np.nan,
                'impro_duration': diff,
                
                'chord_name': get_current_chord(row, linear_chord_progression),
                
                'type': 'improvised'
            })           
             
        ticks = row['ticks']
        duration = row['duration']
        
        if i+duration_correction < impro.shape[0]:
            next_diff = impro.iloc[i+1]['ticks'] - (row['duration'] + row['ticks'])
            if next_diff == duration_correction:
                duration = duration + next_diff
            
        rows.append({
            'diff': diff,
            'ticks': row['ticks'],
            'offset': row['offset'],

            'impro_pitch': row['pitch'],
            'impro_duration': duration,

            'chord_name': row['chord_name'],
            
            'type': 'improvised'
        })
            
    return pd.DataFrame(rows)

In [115]:
mono = extract_mono_time_step_encoding(impro, orig, linear_chord_progression)

Impro WP
Original WP


In [121]:
mono.head(10)

,diff,ticks,offset,impro_pitch,impro_duration,chord_name,type
0,43,0,0,NaN,43,Amin7,improvised
1,43,43,43,69.0,4,Amin7,improvised
2,4,47,47,NaN,4,Amin7,improvised
3,4,51,3,74.0,16,Amin7,improvised
4,0,67,19,72.0,5,Amin7,improvised
5,2,72,24,NaN,2,Amin7,improvised
6,2,74,26,69.0,4,Amin7,improvised
7,2,78,30,NaN,2,Amin7,improvised
8,2,80,32,64.0,6,Amin7,improvised
9,0,86,38,62.0,6,Amin7,improvised


In [122]:
for i in range(mono.shape[0] - 1):
    
    curr_r = mono.iloc[i]
    
    curr_ticks = curr_r['ticks']
    curr_duration = curr_r['impro_duration']
    
    next_r = mono.iloc[i+1]
    
    next_ticks = next_r['ticks']
    
    if int(curr_ticks + curr_duration) != int(next_ticks):
        print('nope', int(curr_ticks + curr_duration), int(next_ticks))
        break

In [124]:
mono[np.isnan(mono['impro_pitch'])]['impro_duration'].value_counts()

2     29
6     14
3      8
4      7
5      6
33     2
7      2
8      2
26     2
11     2
62     1
12     1
24     1
32     1
14     1
34     1
46     1
13     1
43     1
Name: impro_duration, dtype: int64